### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/lower_grade_glioma_and_glioblastoma/GSE28271'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Step 1: Check gene expression data availability
if 'mrna expression' in ["methylation"]:
    is_gene_available = False
else:
    is_gene_available = True

# Step 2: Check variable availability and define data conversion functions

# Sample characteristics dictionary from STEP 1 output
sample_data = {
    0: ['tumor type: Anaplastic', 'tumor type: Glioblastoma', 'tumor type: Lowgrade'],
    1: ['restriction enzyme: HpaII'],
    2: ['idh1 status: MUT', 'idh1 status: WT']
}

# Assume variables are available
trait_row = 0 if 0 in sample_data and len(set(sample_data[0])) > 1 else None
# Age and gender not available from provided data
age_row = None
gender_row = None

# Data conversion functions

def convert_trait(value):
    if value.startswith('tumor type: '):
        label = value.split(': ')[1]
        if label in ['Anaplastic', 'Glioblastoma', 'Lowgrade']:
            return label
    return None

def convert_age(value):
    return None

def convert_gender(value):
    return None

# Save metadata
save_cohort_info('GSE28271', './preprocessed/lower_grade_glioma_and_glioblastoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'lower_grade_glioma_and_glioblastoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/lower_grade_glioma_and_glioblastoma/trait_data/GSE28271.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine which keys store the same kind of identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'ORF'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to convert gene probes into gene symbols
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/lower_grade_glioma_and_glioblastoma/gene_data/GSE28271.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# Ensure data types are appropriate before checking bias
if 'Age' in merged_data.columns:
    merged_data['Age'] = pd.to_numeric(merged_data['Age'], errors='coerce')
if 'Gender' in merged_data.columns:
    merged_data['Gender'] = pd.to_numeric(merged_data['Gender'], errors='coerce')
if 'lower_grade_glioma_and_glioblastoma' in merged_data.columns:
    merged_data['lower_grade_glioma_and_glioblastoma'] = merged_data['lower_grade_glioma_and_glioblastoma'].astype(str)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'lower_grade_glioma_and_glioblastoma')

# 4. Save the cohort information.
trait_biased = trait_biased  # Ensure variable name consistency
save_cohort_info('GSE28271', './preprocessed/lower_grade_glioma_and_glioblastoma/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/lower_grade_glioma_and_glioblastoma/GSE28271.csv'
    unbiased_merged_data.to_csv(csv_path)
